In [ ]:
#bibliotecas

import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#lendo arquivos

df = pd.read_csv(r"C:\Users\anacarol\Desktop\Ic\dados.csv", delimiter = ';')
df

In [ ]:
#inserindo código da ação desejada

codigo_valido = False
while not codigo_valido:
    cod = input("Digite o código da ação desejada: ")
    cod = cod.upper()
    df_cod = df[df['cod_papel'] == cod]
    codigo_valido = bool(df['cod_papel'].str.contains(cod).any())
    if (not codigo_valido):
        print('O código informado não é valido. Digite novamente')
df_cod

In [ ]:
#alterando o tipo de variável

df.dtypes
df['data_pregao'] = pd.to_datetime(df['data_pregao'], format = '%d/%m/%y', exact=False)
df.dtypes

In [ ]:
#criando médias móveis

df_cod['mm5d'] = df_cod['preco_fechamento'].rolling(5).mean()
df_cod['mm21d'] = df_cod['preco_fechamento'].rolling(21).mean()
df_cod

In [ ]:
#empurrando os valores das ações

df_cod['preco_fechamento'] = df_cod['preco_fechamento'].shift(-1)
df_cod

In [ ]:
#retirando dados nulos

df_cod.dropna(inplace=True)
df_cod

In [ ]:
#verificando quantiodade de linhas

qtd_linhas = len(df_cod)
qtd_linhas_treino = round((qtd_linhas * 0.75) - 0.5)
qtd_linhas_teste = round((qtd_linhas * 0.99) - 0.5)

info = {
    f" linhas treino = 0 : {qtd_linhas_treino}"
    f" linhas teste = {qtd_linhas_treino} : {qtd_linhas_teste}"
    f" linhas validação = {qtd_linhas_teste} : {qtd_linhas}"
}

info

In [ ]:
#reindexando data frame

df_cod = df_cod.reset_index(drop = True)
df_cod

In [ ]:
#sepando features e labels

features = df_cod.drop(['cod_papel', 'nome_empresa', 'data_pregao', 'preco_fechamento'], 1)
labels = df_cod['preco_fechamento']

In [ ]:
#Escolhendo as melhores features com Kbest

features_list = ('preco_abertura', 'qtd_neg_efetuados', 'volume_titulos_neg', 'mm5d', 'mm21d')

k_best_features = SelectKBest(k = 'all')
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_
raw_pairs = zip(features_list[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key = lambda x: x[1])))

k_best_features_final = dict(ordered_pairs[:15])
k_best_features = k_best_features_final.keys()
print ('')
print ('Melhores features:')
print (k_best_features_final)

In [ ]:
#separando as features escolhidas

features = df_cod.drop(['cod_papel', 'nome_empresa', 'data_pregao', 'preco_fechamento', 'preco_abertura', 'mm21d'], 1)

In [ ]:
#normalizando os dados de entrada (features)

#gerando novo padrão
scaler = MinMaxScaler().fit(features)
features_scale = scaler.transform(features)

print ('Features: ', features_scale.shape)
print (features_scale)

In [ ]:
#separa os dados de treino, teste e validação

X_train = features_scale[:qtd_linhas_treino]
X_test = features_scale[qtd_linhas_treino:qtd_linhas_teste]

y_train = labels[:qtd_linhas_treino]
y_test = labels[qtd_linhas_treino:qtd_linhas_teste]

print( len(X_train), len(y_train))

print (len(X_test), len(y_test)) 

In [ ]:
#treinamento usando regressão linear

lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
cd = r2_score(y_test, pred)

f'Coeficiência de determinação:{cd * 100:.2f}'

In [ ]:
#rede neural

rn = MLPRegressor(max_iter=2000)

rn.fit(X_train, y_train)
pred = rn.predict(X_test)

cd = rn.score(X_test, y_test)


f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
#rede neural com ajuste hyper parameters

rn = MLPRegressor()

parameter_space = {
    'hidden_layer_sizes': [(i,) for i in list(range(1,21))],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptative'],
}

search = GridSearchCV(rn, parameter_space, n_jobs=-1, cv=5)

search.fit(X_train,y_train)
clf = search.best_estimator_
pred = search.predict(X_test)

cd = search.score(X_test, y_test)

f'Coeficiente de determinação:{cd * 100:.2f}'

In [ ]:
#executando a previsão

previsao = features_scale[qtd_linhas_teste:qtd_linhas]

data_pregao_full = df_cod['data_pregao']
data_pregao = data_pregao_full[qtd_linhas_teste:qtd_linhas]

res_full = df_cod['preco_fechamento']
res = res_full[qtd_linhas_teste:qtd_linhas]

ras_full = df_cod['preco_abertura']
ras = ras_full[qtd_linhas_teste:qtd_linhas]

pred = lr.predict(previsao)

df = pd.DataFrame({'data_pregao':data_pregao, 'abertura':ras,'real':res, 'previsao':pred})
df['real'] = df['real'].shift(+1)

df.set_index('data_pregao', inplace=True)

print(df)


In [ ]:
#verificando se o candle do dia é verde ou vermelho

#valor real
df['comp_real'] = np.where(df['real'] < df['abertura'],'vermelho','verde')

#valor previsto
df['comp_previsao'] = np.where(df['abertura'] < df['previsao'],'verde','vermelho')

#comparação
df['comp'] = np.where(df['comp_previsao'] == df['comp_real'],'igual','diferente')

In [ ]:
#verificando se o dia foi de alta ou baixa na bolsa

#valor real
novas_linhas = []
linha_anterior = df['real'][0]
for linha in df['real']:
    if(linha_anterior != linha):
        if(linha_anterior < linha):
            novas_linhas.append('alto')
        else: 
            novas_linhas.append('baixo')
    linha_anterior = linha
df['verif_real'] = novas_linhas

#valor previsto
novas_linhas = []
linha_anterior = df['previsao'][0]
for linha in df['previsao']:
    if(linha_anterior < linha):
        novas_linhas.append('alto')
    else: 
        novas_linhas.append('baixo')
    linha_anterior = linha
df['verif_previsao'] = novas_linhas

#comparação
df['verif_comp'] = np.where(df['verif_real'] == df['verif_previsao'],'igual','diferente')

In [ ]:
#dataframe final

print(df)

In [ ]:
#calculando desvio padrão

df['diferenca'] = 0.0

total_linhas = range(0,len(df['real']))

for i in total_linhas:
    df['diferenca'][i] = abs(df['real'][i] - df['previsao'][i])
    
print(df['diferenca'].std())